Tôi sẽ giải thích logic mới của API theo phiên bản v2:

### 1. Thay đổi Input/Output Model

```python
# Input có thêm trường robot_type tùy chọn
class InputData(BaseModel):
    robot: str
    user_answer: str
    robot_type: str | None = None  # Optional, mặc định là None

# Output có thêm trường robot_type
class OutputData(BaseModel):
    robot_type: str | None = None
    user_intent: str
    confidence_score: float
    response_time_ms: float
    fast_response: str
```

### 2. Logic xử lý theo robot_type

```python
# Trong hàm predict:
# Xác định robot_type, mặc định là "Workflow"
robot_type = data.robot_type if data.robot_type else "Workflow"

# Chọn fast_response dựa trên robot_type
if robot_type == "Agent":
    fast_response = get_agent_fast_response()  # Dùng responses riêng cho Agent
else:  # Workflow hoặc default
    fast_response = intent_to_fast_response(user_intent)  # Dùng responses dựa trên intent
```

### 3. Các trường hợp xử lý:

1. **Không có robot_type (hoặc Workflow)**:
   ```json
   Input: {"robot": "Câu hỏi", "user_answer": "Câu trả lời"}
   // hoặc
   Input: {"robot": "Câu hỏi", "user_answer": "Câu trả lời", "robot_type": "Workflow"}
   ```
   - Sẽ dùng logic cũ: phân tích intent và trả về fast_response tương ứng với intent đó
   - Response sẽ bao gồm: intent, confidence score, và fast_response dựa trên intent

2. **Có robot_type là "Agent"**:
   ```json
   Input: {"robot": "Câu hỏi", "user_answer": "Câu trả lời", "robot_type": "Agent"}
   ```
   - Vẫn phân tích intent nhưng fast_response sẽ được lấy từ danh sách riêng cho Agent
   - Danh sách Agent responses đơn giản và ngắn gọn hơn
   - Response vẫn bao gồm đầy đủ các trường nhưng fast_response sẽ độc lập với intent

### 4. Ví dụ Response:

```json
// Với Workflow:
{
    "robot_type": "Workflow",
    "user_intent": "intent_neutral",
    "confidence_score": 0.9978,
    "response_time_ms": 24.127,
    "fast_response": "Để Pika nghĩ xem nào*#*#"
}

// Với Agent:
{
    "robot_type": "Agent",
    "user_intent": "intent_negative",
    "confidence_score": 1.00,
    "response_time_ms": 22.5,
    "fast_response": "Để Pika kiểm tra nào"
}
```

Logic mới này cho phép linh hoạt trong việc xử lý fast_response dựa trên loại robot, trong khi vẫn duy trì khả năng phân tích intent cho cả hai trường hợp.
